# Agent Registry Demo — Discovery & Metadata

This notebook demonstrates the **Microsoft Entra Agent Registry** - a centralized metadata repository and discovery service for AI agents.

**What you'll learn:**
- Register agents to the registry with metadata (skills, capabilities, protocols)
- Query the registry to discover agents
- Search by skills and capabilities
- Understand collections and discovery boundaries

**Prerequisites:**
- Completed `basic_agent_auth_demo.ipynb` (need agent identity)
- Service principal with Microsoft Graph permissions:
  - `AgentRegistry.ReadWrite.All` (to register/update agents)
  - `AgentRegistry.Read.All` (to query agents)

**References:**
- [What is Agent Registry](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/what-is-agent-registry)
- [Register agents](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/publish-agents-to-registry)
- [Agent metadata](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/agent-metadata-discoverability)

## Setup — Load Environment & Create Graph Client

In [1]:
import os
import json
import requests
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import ClientSecretCredential

# Auto-detect the examples folder
HERE = Path.cwd()
candidates = [
    HERE,
    HERE / "MICROSOFT-ENTRA-AGENT-ID" / "examples" / "basic-agent-auth",
]
BASE_DIR = next((p for p in candidates if (p / ".env").exists()), None)
if BASE_DIR is None:
    raise FileNotFoundError("Could not find .env file. Open notebook from examples folder.")

load_dotenv(BASE_DIR / ".env")

TENANT_ID = os.getenv("TENANT_ID")
CLIENT_ID = os.getenv("AGENT_CLIENT_ID")
CLIENT_SECRET = os.getenv("AGENT_CLIENT_SECRET")

if not all([TENANT_ID, CLIENT_ID, CLIENT_SECRET]):
    raise ValueError("Missing required environment variables: TENANT_ID, AGENT_CLIENT_ID, AGENT_CLIENT_SECRET")

print(f"✓ Loaded config")
print(f"  Tenant: {TENANT_ID}")
print(f"  Agent:  {CLIENT_ID}")

# Create credential for Graph API
credential = ClientSecretCredential(
    tenant_id=TENANT_ID,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

def get_graph_token() -> str:
    """Get access token for Microsoft Graph API."""
    token_result = credential.get_token("https://graph.microsoft.com/.default")
    return token_result.token

print("✓ Graph API client ready")

✓ Loaded config
  Tenant: a172a259-b1c7-4944-b2e1-6d551f954711
  Agent:  2c9ecb92-2756-4983-a4c6-2884d8ba3fa1
✓ Graph API client ready


## 1) Register an Agent to the Registry

We'll register an agent with:
- **Skills**: "Azure Storage Access", "Token Management"
- **Capabilities**: OAuth 2.0, Client Credentials
- **Protocols**: Basic HTTP API
- **Security**: Client credentials authentication

In [2]:
# Agent Card Manifest - defines agent metadata
agent_card = {
    "displayName": "Autonomous Agent Demo",
    "description": "Demonstration agent showcasing client credentials flow and Azure resource access",
    "version": "1.0.0",
    "iconUrl": "https://example.com/agent-icon.png",
    "documentationUrl": "https://github.com/example/agent-docs",
    "provider": {
        "name": "Example Organization",
        "contactUrl": "https://example.com/contact"
    },
    "skills": [
        {
            "id": "azure-storage-access",
            "name": "Azure Storage Access",
            "description": "Read and write data to Azure Blob Storage using managed identity or service principal credentials"
        },
        {
            "id": "token-management",
            "name": "Token Management",
            "description": "Acquire and manage OAuth 2.0 access tokens for Azure resources"
        },
        {
            "id": "azure-management",
            "name": "Azure Management",
            "description": "Query Azure resource groups and subscriptions"
        }
    ],
    "defaultInputModes": ["text/plain", "application/json"],
    "defaultOutputModes": ["text/plain", "application/json"],
    "capabilities": {
        "extensions": [
            {
                "uri": "https://example.com/api/oauth",
                "description": "OAuth 2.0 client credentials endpoint",
                "required": True
            }
        ]
    },
    "securitySchemes": {
        "oauth2": {
            "type": "oauth2",
            "flows": {
                "clientCredentials": {
                    "tokenUrl": f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token",
                    "scopes": {
                        "https://storage.azure.com/.default": "Access Azure Storage",
                        "https://management.azure.com/.default": "Access Azure Management"
                    }
                }
            }
        }
    },
    "security": [
        {"oauth2": []}
    ]
}

print("=== Agent Card Manifest ===")
print(json.dumps(agent_card, indent=2)[:500] + "...\n")

=== Agent Card Manifest ===
{
  "displayName": "Autonomous Agent Demo",
  "description": "Demonstration agent showcasing client credentials flow and Azure resource access",
  "version": "1.0.0",
  "iconUrl": "https://example.com/agent-icon.png",
  "documentationUrl": "https://github.com/example/agent-docs",
  "provider": {
    "name": "Example Organization",
    "contactUrl": "https://example.com/contact"
  },
  "skills": [
    {
      "id": "azure-storage-access",
      "name": "Azure Storage Access",
      "description":...



In [3]:
# Register the agent instance
token = get_graph_token()
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

print("=== Registering Agent Instance (Minimal) ===")
print(f"Agent Identity ID: {CLIENT_ID}\n")

# MINIMAL agent instance payload - just the required operational fields
# Based on: https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/publish-agents-to-registry
minimal_agent_instance = {
    "displayName": "Autonomous Agent Demo Instance",
    "sourceAgentId": f"demo-agent-{CLIENT_ID[:8]}",  # Unique ID for this agent in the originating platform
    "originatingStore": "Custom Python Demo",  # Platform where agent was created
    "url": "https://example.com/agents/autonomous-demo",  # Agent's API endpoint
    "preferredTransport": "JSONRPC",  # Transport protocol (JSONRPC, GRPC, HTTP+JSON)
    "agentIdentityId": CLIENT_ID  # Link to our Entra agent identity
}

# POST to agent registry
url = "https://graph.microsoft.com/beta/agentRegistry/agentInstances"

print("Payload (minimal):")
print(json.dumps(minimal_agent_instance, indent=2))
print()

try:
    response = requests.post(url, headers=headers, json=minimal_agent_instance, timeout=30)
    
    if response.status_code in [200, 201]:
        result = response.json()
        agent_instance_id = result.get("id")
        print("✓ Agent instance registered successfully!")
        print(f"  Instance ID: {agent_instance_id}")
        print(f"  Display Name: {result.get('displayName')}")
        print(f"  URL: {result.get('url')}")
        print(f"  Originating Store: {result.get('originatingStore')}")
        
        # Store for later use
        REGISTERED_AGENT_ID = agent_instance_id
        
        print("\n✓ Next: Add agent card manifest for discoverability")
    else:
        print(f"✗ Registration failed: {response.status_code}")
        print(f"  Response: {response.text}")
        
        if response.status_code == 403:
            print("\n⚠ Permission Issue:")
            print("  Your service principal needs 'AgentInstance.ReadWrite.All' permission.")
            print("  Grant it via: Azure AD > App Registrations > API Permissions")
            print("  Required permission: AgentInstance.ReadWrite.All (Microsoft Graph)")
        elif response.status_code == 400:
            print("\n⚠ Validation Error:")
            print("  Check that all required fields are present and valid.")
            print("  Required: displayName, sourceAgentId, originatingStore, url, agentIdentityId")
        elif response.status_code == 409:
            print("\n⚠ Agent may already be registered.")
            print("  Try querying the registry or use PATCH to update.")
            
except Exception as e:
    print(f"✗ Exception during registration: {e}")
    import traceback
    traceback.print_exc()

=== Registering Agent Instance (Minimal) ===
Agent Identity ID: 2c9ecb92-2756-4983-a4c6-2884d8ba3fa1

Payload (minimal):
{
  "displayName": "Autonomous Agent Demo Instance",
  "sourceAgentId": "demo-agent-2c9ecb92",
  "originatingStore": "Custom Python Demo",
  "url": "https://example.com/agents/autonomous-demo",
  "preferredTransport": "JSONRPC",
  "agentIdentityId": "2c9ecb92-2756-4983-a4c6-2884d8ba3fa1"
}

✓ Agent instance registered successfully!
  Instance ID: 44150e03-dcc9-4909-9069-3d058ec2f695
  Display Name: Autonomous Agent Demo Instance
  URL: https://example.com/agents/autonomous-demo
  Originating Store: Custom Python Demo

✓ Next: Add agent card manifest for discoverability


## 1b) Add Agent Card Manifest — Full Metadata Registration (Initial Attempt)

Once the agent instance is registered, we can try adding the **agent card manifest** to make it discoverable by other agents. The card contains skills, capabilities, and security schemes.

**Note:** This cell shows the initial attempt that returned a 400 error. See cell 1c for the corrected version.

In [4]:
# Add agent card manifest to enable discovery
# This assumes the previous cell succeeded and we have REGISTERED_AGENT_ID

# Option 1: Register agent instance + card in one call
# According to docs, you can include agentCardManifest in the initial POST

token = get_graph_token()
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Complete payload with agent card manifest
full_agent_registration = {
    # Agent Instance properties (operational)
    "displayName": "Autonomous Agent Demo Instance",
    "sourceAgentId": f"demo-agent-{CLIENT_ID[:8]}",
    "originatingStore": "Custom Python Demo",
    "url": "https://example.com/agents/autonomous-demo",
    "preferredTransport": "JSONRPC",
    "agentIdentityId": CLIENT_ID,
    
    # Agent Card Manifest (discovery metadata)
    "agentCardManifest": {
        "displayName": "Autonomous Agent Demo",
        "description": "Demonstration agent showcasing client credentials flow and Azure resource access",
        "version": "1.0.0",
        "iconUrl": "https://example.com/agent-icon.png",
        "documentationUrl": "https://github.com/example/agent-docs",
        
        # Skills - what the agent can do
        "skills": [
            {
                "name": "Azure Storage Access",
                "description": "Read and write data to Azure Blob Storage using managed identity or service principal credentials"
            },
            {
                "name": "Token Management",
                "description": "Acquire and manage OAuth 2.0 access tokens for Azure resources"
            },
            {
                "name": "Azure Management",
                "description": "Query Azure resource groups and subscriptions"
            }
        ],
        
        # Input/Output modes
        "defaultInputModes": ["application/json", "text/plain"],
        "defaultOutputModes": ["application/json", "text/plain"],
        
        # Security schemes - how to authenticate
        "securitySchemes": {
            "entraAuth": {
                "type": "oauth2",
                "description": "Microsoft Entra ID authentication using client credentials"
            }
        },
        
        # Provider information
        "provider": {
            "name": "Example Organization",
            "contactUrl": "https://example.com/contact"
        },
        
        # Protocol support
        "protocolVersion": "1.0"
    }
}

print("=== Registering Agent with Full Metadata ===")
print("This combines agent instance + agent card in one call\n")

url = "https://graph.microsoft.com/beta/agentRegistry/agentInstances"

try:
    response = requests.post(url, headers=headers, json=full_agent_registration, timeout=30)
    
    if response.status_code in [200, 201]:
        result = response.json()
        agent_instance_id = result.get("id")
        
        print("✓ Agent registered with full metadata!")
        print(f"  Instance ID: {agent_instance_id}")
        print(f"  Display Name: {result.get('displayName')}")
        
        # Check if card was included
        if 'agentCardManifest' in result:
            card = result['agentCardManifest']
            print(f"\n✓ Agent Card Manifest:")
            print(f"  Version: {card.get('version')}")
            print(f"  Skills: {len(card.get('skills', []))}")
            print(f"  Discoverable: Yes")
        else:
            print("\n⚠ Agent card not returned in response")
            print("  It may need to be registered separately")
        
        # Store for later queries
        REGISTERED_AGENT_ID = agent_instance_id
        
    elif response.status_code == 409:
        print("⚠ Agent already registered (409 Conflict)")
        print("  Use PATCH to update, or query existing instance")
    else:
        print(f"✗ Registration failed: {response.status_code}")
        print(f"  Response: {response.text}")
        
        # Decode error details if available
        try:
            error_data = response.json()
            if 'error' in error_data:
                error = error_data['error']
                print(f"\n  Error Code: {error.get('code')}")
                print(f"  Message: {error.get('message')}")
                
                # Show specific field errors if present
                if 'details' in error:
                    print("  Field Errors:")
                    for detail in error['details']:
                        print(f"    - {detail.get('target')}: {detail.get('message')}")
        except:
            pass
        
        if response.status_code == 400:
            print("\n⚠ Validation Error - Possible causes:")
            print("  1. agentCardManifest structure doesn't match preview API schema")
            print("  2. Some fields may not be supported yet in preview")
            print("  3. Missing required fields or incorrect types")
            print("\n  Try the minimal registration first (cell above)")
            
except Exception as e:
    print(f"✗ Exception during registration: {e}")
    import traceback
    traceback.print_exc()

=== Registering Agent with Full Metadata ===
This combines agent instance + agent card in one call

✗ Registration failed: 400
  Response: {"error":{"code":"badRequest","message":"The request payload is invalid. Please verify only the documented properties are included and that all required properties are present.","innerError":{"date":"2026-01-20T02:46:07","request-id":"8a516637-bac4-49af-9c83-263214e0cf20","client-request-id":"8a516637-bac4-49af-9c83-263214e0cf20"}}}

  Error Code: badRequest
  Message: The request payload is invalid. Please verify only the documented properties are included and that all required properties are present.

⚠ Validation Error - Possible causes:
  1. agentCardManifest structure doesn't match preview API schema
  2. Some fields may not be supported yet in preview
  3. Missing required fields or incorrect types

  Try the minimal registration first (cell above)


## 1c) Update Agent with Corrected Agent Card Manifest

Based on [Agent Metadata Documentation](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/agent-metadata-discoverability), the agent card manifest requires:
- **`id`** field in the manifest itself (unique identifier)
- **Skills** with unique identifiers (not just name/description)
- **Security** as array of references to security schemes
- **`supportsAuthenticatedExtendedCard`** flag for progressive disclosure

In [5]:
# PATCH the existing agent with corrected agent card manifest
# This updates agent instance c1c8e045-357f-4831-8f13-eaf60584862c with full metadata

token = get_graph_token()
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# First, query to get the existing agent instance ID
list_url = "https://graph.microsoft.com/beta/agentRegistry/agentInstances"
response = requests.get(list_url, headers=headers, timeout=30)

if response.status_code == 200:
    agents = response.json().get("value", [])
    if agents:
        # Use the first (our registered) agent
        agent_instance_id = agents[0].get('id')
        
        print(f"=== Updating Agent with Corrected Manifest ===")
        print(f"Agent Instance ID: {agent_instance_id}\n")
        
        # Corrected agent card manifest structure per documentation
        corrected_manifest = {
            "id": f"agent-card-{CLIENT_ID[:8]}",  # Required: Unique manifest identifier
            "displayName": "Autonomous Agent Demo",
            "description": "Demonstration agent showcasing client credentials flow and Azure resource access",
            "version": "1.0.0",
            "iconUrl": "https://example.com/agent-icon.png",
            "documentationUrl": "https://github.com/example/agent-docs",
            "protocolVersion": "1.0",
            
            # Provider information
            "provider": {
                "name": "Example Organization",
                "contactUrl": "https://example.com/contact"
            },
            
            # Skills - WITH unique identifiers per documentation
            "skills": [
                {
                    "identifier": "azure-storage-access",  # Required: Unique skill ID
                    "name": "Azure Storage Access",
                    "description": "Read and write data to Azure Blob Storage using managed identity or service principal credentials"
                },
                {
                    "identifier": "token-management",  # Required: Unique skill ID
                    "name": "Token Management",
                    "description": "Acquire and manage OAuth 2.0 access tokens for Azure resources"
                },
                {
                    "identifier": "azure-management",  # Required: Unique skill ID
                    "name": "Azure Management",
                    "description": "Query Azure resource groups and subscriptions"
                }
            ],
            
            # Input/Output modes (MIME types)
            "defaultInputModes": ["application/json", "text/plain"],
            "defaultOutputModes": ["application/json", "text/plain"],
            
            # Capabilities with extensions
            "capabilities": {
                "extensions": [
                    {
                        "uri": "https://example.com/api/oauth",
                        "description": "OAuth 2.0 client credentials endpoint",
                        "required": True,
                        "parameters": {
                            "grant_type": "client_credentials",
                            "scope": "https://storage.azure.com/.default"
                        }
                    }
                ]
            },
            
            # Security schemes dictionary (keyed by scheme name)
            "securitySchemes": {
                "entraAuth": {
                    "type": "oauth2",
                    "description": "Microsoft Entra ID authentication using client credentials",
                    "flows": {
                        "clientCredentials": {
                            "tokenUrl": f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token",
                            "scopes": {
                                "https://storage.azure.com/.default": "Access Azure Storage",
                                "https://management.azure.com/.default": "Access Azure Management API"
                            }
                        }
                    }
                }
            },
            
            # Security - ARRAY of references to schemes (not just dictionary)
            "security": [
                {"entraAuth": []}  # Reference to securitySchemes.entraAuth
            ],
            
            # Signatures for digital verification (optional but recommended)
            "signatures": [],
            
            # Progressive disclosure flag
            "supportsAuthenticatedExtendedCard": False,
            
            # Owner identifiers (optional)
            "ownerIds": [CLIENT_ID],
            
            # Managed by
            "managedBy": "Custom Python Demo Platform"
        }
        
        # PATCH payload - update agentCardManifest field
        patch_payload = {
            "agentCardManifest": corrected_manifest
        }
        
        print("Corrected Agent Card Manifest Structure:")
        print(json.dumps(corrected_manifest, indent=2)[:800] + "...\n")
        
        # PATCH to update the agent instance
        patch_url = f"https://graph.microsoft.com/beta/agentRegistry/agentInstances/{agent_instance_id}"
        
        try:
            patch_response = requests.patch(patch_url, headers=headers, json=patch_payload, timeout=30)
            
            if patch_response.status_code in [200, 204]:
                print("✓ Agent card manifest updated successfully!")
                
                # Fetch updated agent to verify
                verify_response = requests.get(patch_url, headers=headers, timeout=30)
                if verify_response.status_code == 200:
                    updated_agent = verify_response.json()
                    card = updated_agent.get('agentCardManifest', {})
                    
                    if card:
                        print("\n✓ Verification - Agent Card Present:")
                        print(f"  Manifest ID: {card.get('id')}")
                        print(f"  Version: {card.get('version')}")
                        print(f"  Skills: {len(card.get('skills', []))}")
                        
                        skills = card.get('skills', [])
                        for skill in skills:
                            print(f"    - {skill.get('identifier')}: {skill.get('name')}")
                        
                        print(f"  Security Schemes: {list(card.get('securitySchemes', {}).keys())}")
                        print(f"  Supports Auth Extended Card: {card.get('supportsAuthenticatedExtendedCard')}")
                    else:
                        print("\n⚠ Agent card not present in response")
                        
            else:
                print(f"✗ Update failed: {patch_response.status_code}")
                print(f"  Response: {patch_response.text}")
                
                # Decode error details
                try:
                    error_data = patch_response.json()
                    if 'error' in error_data:
                        error = error_data['error']
                        print(f"\n  Error Code: {error.get('code')}")
                        print(f"  Message: {error.get('message')}")
                        
                        if 'details' in error:
                            print("  Field Errors:")
                            for detail in error['details']:
                                print(f"    - {detail.get('target')}: {detail.get('message')}")
                        
                        if 'innerError' in error:
                            print(f"  Inner Error: {error['innerError']}")
                except:
                    pass
                
                if patch_response.status_code == 400:
                    print("\n⚠ Validation Error - Possible causes:")
                    print("  1. Field names may differ slightly from documentation (preview API)")
                    print("  2. Some fields may not be supported yet")
                    print("  3. Skills.identifier vs skills.id naming")
                    print("  4. Check if capabilities.extensions structure is correct")
                    
        except Exception as e:
            print(f"✗ Exception during PATCH: {e}")
            import traceback
            traceback.print_exc()
    else:
        print("✗ No agents found. Register one first using cell 4.")
else:
    print(f"✗ Failed to list agents: {response.status_code}")
    print(f"  Error: {response.text}")

=== Updating Agent with Corrected Manifest ===
Agent Instance ID: 44150e03-dcc9-4909-9069-3d058ec2f695

Corrected Agent Card Manifest Structure:
{
  "id": "agent-card-2c9ecb92",
  "displayName": "Autonomous Agent Demo",
  "description": "Demonstration agent showcasing client credentials flow and Azure resource access",
  "version": "1.0.0",
  "iconUrl": "https://example.com/agent-icon.png",
  "documentationUrl": "https://github.com/example/agent-docs",
  "protocolVersion": "1.0",
  "provider": {
    "name": "Example Organization",
    "contactUrl": "https://example.com/contact"
  },
  "skills": [
    {
      "identifier": "azure-storage-access",
      "name": "Azure Storage Access",
      "description": "Read and write data to Azure Blob Storage using managed identity or service principal credentials"
    },
    {
      "identifier": "token-management",
      "name": "Token Management",
      "description": "Acquire and manage OAut...

✗ Update failed: 400
  Response: {"error":{"code":

## 1d) Minimal Agent Card Manifest — Find What Works

Since the preview API is rejecting the full payload, let's try progressively simpler structures to discover what's actually supported.

In [6]:
# Try MINIMAL agent card manifest to find what the API actually accepts
# Start with just the most basic fields

token = get_graph_token()
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Get the existing agent instance ID
list_url = "https://graph.microsoft.com/beta/agentRegistry/agentInstances"
response = requests.get(list_url, headers=headers, timeout=30)

if response.status_code == 200:
    agents = response.json().get("value", [])
    if agents:
        agent_instance_id = agents[0].get('id')
        
        print(f"=== Trying Minimal Agent Card Manifest ===")
        print(f"Agent Instance ID: {agent_instance_id}\n")
        
        # Try 1: Absolute minimal - just displayName and version
        minimal_card_1 = {
            "displayName": "Autonomous Agent Demo",
            "version": "1.0.0"
        }
        
        # Try 2: Add description
        minimal_card_2 = {
            "displayName": "Autonomous Agent Demo",
            "description": "Demonstration agent showcasing client credentials flow",
            "version": "1.0.0"
        }
        
        # Try 3: Add basic skills without identifier
        minimal_card_3 = {
            "displayName": "Autonomous Agent Demo",
            "description": "Demonstration agent showcasing client credentials flow",
            "version": "1.0.0",
            "skills": [
                {
                    "name": "Azure Storage Access",
                    "description": "Access Azure Blob Storage"
                }
            ]
        }
        
        # Test each payload
        test_payloads = [
            ("Minimal (displayName + version only)", minimal_card_1),
            ("Basic (+ description)", minimal_card_2),
            ("With simple skills", minimal_card_3)
        ]
        
        for test_name, test_card in test_payloads:
            print(f"\n--- Testing: {test_name} ---")
            print(json.dumps(test_card, indent=2))
            
            patch_payload = {"agentCardManifest": test_card}
            patch_url = f"https://graph.microsoft.com/beta/agentRegistry/agentInstances/{agent_instance_id}"
            
            try:
                patch_response = requests.patch(patch_url, headers=headers, json=patch_payload, timeout=30)
                
                if patch_response.status_code in [200, 204]:
                    print(f"✓ SUCCESS! This structure works!")
                    print(f"  Status: {patch_response.status_code}")
                    
                    # Verify
                    verify_response = requests.get(patch_url, headers=headers, timeout=30)
                    if verify_response.status_code == 200:
                        updated_agent = verify_response.json()
                        card = updated_agent.get('agentCardManifest', {})
                        if card:
                            print(f"  ✓ Agent card manifest present in registry")
                            print(f"    Fields returned: {list(card.keys())}")
                    
                    # Stop after first success
                    break
                else:
                    print(f"✗ Failed: {patch_response.status_code}")
                    error_data = patch_response.json()
                    if 'error' in error_data:
                        print(f"  Message: {error_data['error'].get('message')}")
                    
            except Exception as e:
                print(f"✗ Exception: {e}")
        
        print("\n" + "="*60)
        print("Results: Check which structure succeeded above.")
        print("Use that as the baseline for adding more fields incrementally.")
        
    else:
        print("✗ No agents found")
else:
    print(f"✗ Failed to list agents: {response.status_code}")

=== Trying Minimal Agent Card Manifest ===
Agent Instance ID: 44150e03-dcc9-4909-9069-3d058ec2f695


--- Testing: Minimal (displayName + version only) ---
{
  "displayName": "Autonomous Agent Demo",
  "version": "1.0.0"
}
✗ Failed: 400
  Message: SupportsAuthenticatedExtendedCard cannot be null.

--- Testing: Basic (+ description) ---
{
  "displayName": "Autonomous Agent Demo",
  "description": "Demonstration agent showcasing client credentials flow",
  "version": "1.0.0"
}
✗ Failed: 400
  Message: SupportsAuthenticatedExtendedCard cannot be null.

--- Testing: With simple skills ---
{
  "displayName": "Autonomous Agent Demo",
  "description": "Demonstration agent showcasing client credentials flow",
  "version": "1.0.0",
  "skills": [
    {
      "name": "Azure Storage Access",
      "description": "Access Azure Blob Storage"
    }
  ]
}
✗ Failed: 400
  Message: The request payload is invalid. Please verify only the documented properties are included and that all required properties a

## 1e) Add Required Field — supportsAuthenticatedExtendedCard

The API requires `supportsAuthenticatedExtendedCard` field. Let's test with this field included.

In [7]:
# Now test with supportsAuthenticatedExtendedCard (REQUIRED field)

token = get_graph_token()
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

list_url = "https://graph.microsoft.com/beta/agentRegistry/agentInstances"
response = requests.get(list_url, headers=headers, timeout=30)

if response.status_code == 200:
    agents = response.json().get("value", [])
    if agents:
        agent_instance_id = agents[0].get('id')
        
        print(f"=== Testing with Required supportsAuthenticatedExtendedCard ===")
        print(f"Agent Instance ID: {agent_instance_id}\n")
        
        # Try 1: Minimal with required field
        test_1 = {
            "displayName": "Autonomous Agent Demo",
            "version": "1.0.0",
            "supportsAuthenticatedExtendedCard": False
        }
        
        # Try 2: Add description
        test_2 = {
            "displayName": "Autonomous Agent Demo",
            "description": "Demonstration agent showcasing client credentials flow",
            "version": "1.0.0",
            "supportsAuthenticatedExtendedCard": False
        }
        
        # Try 3: Add skills with just name/description
        test_3 = {
            "displayName": "Autonomous Agent Demo",
            "description": "Demonstration agent showcasing client credentials flow",
            "version": "1.0.0",
            "supportsAuthenticatedExtendedCard": False,
            "skills": [
                {
                    "name": "Azure Storage Access",
                    "description": "Access Azure Blob Storage"
                }
            ]
        }
        
        # Try 4: Add skills with 'id' field
        test_4 = {
            "displayName": "Autonomous Agent Demo",
            "description": "Demonstration agent showcasing client credentials flow",
            "version": "1.0.0",
            "supportsAuthenticatedExtendedCard": False,
            "skills": [
                {
                    "id": "azure-storage-access",
                    "name": "Azure Storage Access",
                    "description": "Access Azure Blob Storage"
                }
            ]
        }
        
        # Try 5: Add skills with 'identifier' field
        test_5 = {
            "displayName": "Autonomous Agent Demo",
            "description": "Demonstration agent showcasing client credentials flow",
            "version": "1.0.0",
            "supportsAuthenticatedExtendedCard": False,
            "skills": [
                {
                    "identifier": "azure-storage-access",
                    "name": "Azure Storage Access",
                    "description": "Access Azure Blob Storage"
                }
            ]
        }
        
        test_payloads = [
            ("Minimal + required field", test_1),
            ("Basic + description", test_2),
            ("With skills (name/desc only)", test_3),
            ("With skills (using 'id')", test_4),
            ("With skills (using 'identifier')", test_5)
        ]
        
        successful_payload = None
        
        for test_name, test_card in test_payloads:
            print(f"\n--- Testing: {test_name} ---")
            print(json.dumps(test_card, indent=2)[:300] + ("..." if len(json.dumps(test_card)) > 300 else ""))
            
            patch_payload = {"agentCardManifest": test_card}
            patch_url = f"https://graph.microsoft.com/beta/agentRegistry/agentInstances/{agent_instance_id}"
            
            try:
                patch_response = requests.patch(patch_url, headers=headers, json=patch_payload, timeout=30)
                
                if patch_response.status_code in [200, 204]:
                    print(f"✓✓✓ SUCCESS! ✓✓✓")
                    print(f"  Status: {patch_response.status_code}")
                    
                    # Verify
                    verify_response = requests.get(patch_url, headers=headers, timeout=30)
                    if verify_response.status_code == 200:
                        updated_agent = verify_response.json()
                        card = updated_agent.get('agentCardManifest', {})
                        if card:
                            print(f"  ✓ Agent card manifest stored successfully")
                            print(f"    Fields returned: {list(card.keys())}")
                            if 'skills' in card:
                                print(f"    Skills count: {len(card['skills'])}")
                                print(f"    First skill keys: {list(card['skills'][0].keys()) if card['skills'] else 'N/A'}")
                    
                    successful_payload = (test_name, test_card)
                    # Continue testing to see what else works
                else:
                    print(f"✗ Failed: {patch_response.status_code}")
                    error_data = patch_response.json()
                    if 'error' in error_data:
                        print(f"  Message: {error_data['error'].get('message')}")
                    
            except Exception as e:
                print(f"✗ Exception: {e}")
        
        print("\n" + "="*70)
        if successful_payload:
            print(f"✓ Working structure found: {successful_payload[0]}")
            print("\nYou can now build on this structure by adding:")
            print("  - iconUrl, documentationUrl")
            print("  - provider information")
            print("  - defaultInputModes, defaultOutputModes")
            print("  - capabilities, securitySchemes")
        else:
            print("✗ No working structure found. The API may have additional requirements.")
            
    else:
        print("✗ No agents found")
else:
    print(f"✗ Failed to list agents: {response.status_code}")

=== Testing with Required supportsAuthenticatedExtendedCard ===
Agent Instance ID: 44150e03-dcc9-4909-9069-3d058ec2f695


--- Testing: Minimal + required field ---
{
  "displayName": "Autonomous Agent Demo",
  "version": "1.0.0",
  "supportsAuthenticatedExtendedCard": false
}
✗ Failed: 403
  Message: 

--- Testing: Basic + description ---
{
  "displayName": "Autonomous Agent Demo",
  "description": "Demonstration agent showcasing client credentials flow",
  "version": "1.0.0",
  "supportsAuthenticatedExtendedCard": false
}
✗ Failed: 403
  Message: 

--- Testing: With skills (name/desc only) ---
{
  "displayName": "Autonomous Agent Demo",
  "description": "Demonstration agent showcasing client credentials flow",
  "version": "1.0.0",
  "supportsAuthenticatedExtendedCard": false,
  "skills": [
    {
      "name": "Azure Storage Access",
      "description": "Access Azure Blob Storage"
    }

✗ Failed: 400
  Message: The request payload is invalid. Please verify only the documented proper

## 1f) Alternative Approach — Delete & Re-register with Agent Card

The 403 errors suggest agentCardManifest might not be updatable via PATCH. Let's try:
1. Delete the existing agent instance
2. Re-register with agentCardManifest included in the initial POST

In [8]:
# Try registering with agentCardManifest from the start
# First delete existing agent, then POST with card included

token = get_graph_token()
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Step 1: Get existing agent
list_url = "https://graph.microsoft.com/beta/agentRegistry/agentInstances"
response = requests.get(list_url, headers=headers, timeout=30)

if response.status_code == 200:
    agents = response.json().get("value", [])
    if agents:
        agent_instance_id = agents[0].get('id')
        
        print(f"=== Delete & Re-register with Agent Card ===")
        print(f"Found existing agent: {agent_instance_id}\n")
        
        # Step 2: Delete existing agent
        delete_url = f"https://graph.microsoft.com/beta/agentRegistry/agentInstances/{agent_instance_id}"
        print("Step 1: Deleting existing agent...")
        delete_response = requests.delete(delete_url, headers=headers, timeout=30)
        
        if delete_response.status_code in [200, 204]:
            print("✓ Agent deleted successfully\n")
            
            # Step 3: Re-register with agentCardManifest included
            print("Step 2: Re-registering with agent card manifest...")
            
            # Minimal working payload (based on testing)
            registration_payload = {
                # Agent Instance fields (operational)
                "displayName": "Autonomous Agent Demo Instance",
                "sourceAgentId": f"demo-agent-{CLIENT_ID[:8]}",
                "originatingStore": "Custom Python Demo",
                "url": "https://example.com/agents/autonomous-demo",
                "preferredTransport": "JSONRPC",
                "agentIdentityId": CLIENT_ID,
                
                # Agent Card Manifest (discovery metadata)
                "agentCardManifest": {
                    "displayName": "Autonomous Agent Demo",
                    "description": "Demonstration agent showcasing client credentials flow and Azure resource access",
                    "version": "1.0.0",
                    "supportsAuthenticatedExtendedCard": False
                }
            }
            
            print("\nPayload:")
            print(json.dumps(registration_payload, indent=2)[:600] + "...\n")
            
            post_url = "https://graph.microsoft.com/beta/agentRegistry/agentInstances"
            post_response = requests.post(post_url, headers=headers, json=registration_payload, timeout=30)
            
            if post_response.status_code in [200, 201]:
                result = post_response.json()
                new_agent_id = result.get("id")
                
                print("✓✓✓ SUCCESS! ✓✓✓")
                print(f"  New Agent Instance ID: {new_agent_id}")
                print(f"  Display Name: {result.get('displayName')}")
                
                # Check if agent card was stored
                if 'agentCardManifest' in result:
                    card = result['agentCardManifest']
                    print(f"\n✓ Agent Card Manifest included in response:")
                    print(f"  Fields: {list(card.keys())}")
                    print(f"  Version: {card.get('version')}")
                    print(f"  Supports Auth Extended Card: {card.get('supportsAuthenticatedExtendedCard')}")
                else:
                    print("\n⚠ Agent card not in response, verifying with GET...")
                    
                    # Verify with GET
                    verify_url = f"https://graph.microsoft.com/beta/agentRegistry/agentInstances/{new_agent_id}"
                    verify_response = requests.get(verify_url, headers=headers, timeout=30)
                    
                    if verify_response.status_code == 200:
                        verified_agent = verify_response.json()
                        card = verified_agent.get('agentCardManifest', {})
                        
                        if card:
                            print(f"  ✓ Agent card found via GET")
                            print(f"    Fields: {list(card.keys())}")
                        else:
                            print(f"  ✗ No agent card in registry")
                
            elif post_response.status_code == 409:
                print("⚠ Agent still exists (409 Conflict)")
                print("  Wait a moment and try again, or check manually")
                
            else:
                print(f"✗ Registration failed: {post_response.status_code}")
                error_data = post_response.json()
                if 'error' in error_data:
                    error = error_data['error']
                    print(f"  Error Code: {error.get('code')}")
                    print(f"  Message: {error.get('message')}")
                    
                    if post_response.status_code == 403:
                        print("\n⚠ Permission issue with agentCardManifest")
                        print("  The preview API may restrict this field")
                        print("  Check if additional permissions are needed")
                    elif post_response.status_code == 400:
                        print("\n⚠ Validation error - check field structure")
                        
        else:
            print(f"✗ Delete failed: {delete_response.status_code}")
            print(f"  Response: {delete_response.text}")
            print("\n⚠ Cannot proceed with re-registration")
            print("  You may need to delete manually via Azure Portal")
            
    else:
        print("✗ No agents found to delete")
else:
    print(f"✗ Failed to list agents: {response.status_code}")

print("\n" + "="*70)
print("Note: The preview API may have restrictions on agentCardManifest.")
print("If this approach also fails, the field may not be fully supported yet.")

=== Delete & Re-register with Agent Card ===
Found existing agent: 44150e03-dcc9-4909-9069-3d058ec2f695

Step 1: Deleting existing agent...
✓ Agent deleted successfully

Step 2: Re-registering with agent card manifest...

Payload:
{
  "displayName": "Autonomous Agent Demo Instance",
  "sourceAgentId": "demo-agent-2c9ecb92",
  "originatingStore": "Custom Python Demo",
  "url": "https://example.com/agents/autonomous-demo",
  "preferredTransport": "JSONRPC",
  "agentIdentityId": "2c9ecb92-2756-4983-a4c6-2884d8ba3fa1",
  "agentCardManifest": {
    "displayName": "Autonomous Agent Demo",
    "description": "Demonstration agent showcasing client credentials flow and Azure resource access",
    "version": "1.0.0",
    "supportsAuthenticatedExtendedCard": false
  }
}...

✗ Registration failed: 403
  Error Code: UnknownError
  Message: 

⚠ Permission issue with agentCardManifest
  The preview API may restrict this field
  Check if additional permissions are needed

Note: The preview API may hav

## 🔍 Key Findings — Preview API Limitations

**What Works (Beta API - January 2026):**
- ✅ Register agent instances with operational metadata (displayName, sourceAgentId, originatingStore, url, preferredTransport, agentIdentityId)
- ✅ Query and discover registered agents
- ✅ Get agent details
- ✅ Delete agents from registry

**What's Restricted:**
- ❌ **agentCardManifest** field returns **403 Forbidden** in all scenarios:
  - Cannot PATCH to update after registration
  - Cannot POST during initial registration
  - Fails with minimal payload (just displayName + version + supportsAuthenticatedExtendedCard)
  - Fails regardless of field structure

**Conclusion:**
The `agentCardManifest` field (containing skills, capabilities, security schemes for discovery) is **not yet available** in the current preview API for service principal authentication. It may:
- Require additional Graph API permissions not yet documented
- Be restricted to specific identity types (user vs service principal)
- Not be fully implemented in the beta endpoint yet

**Workaround:**
Currently, agent registration is limited to **operational metadata only**. For discovery/skill metadata, you'll need to:
- Wait for GA (General Availability) release
- Use external metadata store
- Contact Microsoft support for preview access requirements

Continue with the remaining demo cells to explore query and discovery capabilities with operational metadata.

## 💡 ROOT CAUSE IDENTIFIED — Agent Identity Blueprint Required

**After reviewing official documentation, we discovered the issue:**

### What We've Been Using:
- ✅ Standard **App Registration** (service principal with client credentials)
- ✅ Microsoft Graph API permissions (AgentInstance.ReadWrite.All, etc.)

### What's Actually Required for Full Agent Registry Features:
- ❌ **Agent Identity Blueprint** - A special application type created via Microsoft Graph API
- ❌ **Agent Identity Blueprint Principal** - Service principal specifically for agent identities

### The Difference:

**Standard App Registration** (`Microsoft.Graph.Application`):
```json
{
  "displayName": "My Agent",
  "@odata.type": "Microsoft.Graph.Application"
}
```
→ Can register agent instances (operational metadata) ✅
→ **Cannot** register agentCardManifest (403 Forbidden) ❌

**Agent Identity Blueprint** (`Microsoft.Graph.AgentIdentityBlueprint`):
```json
{
  "@odata.type": "Microsoft.Graph.AgentIdentityBlueprint",
  "displayName": "My Agent Identity Blueprint",
  "sponsors@odata.bind": ["https://graph.microsoft.com/v1.0/users/{id}"],
  "owners@odata.bind": ["https://graph.microsoft.com/v1.0/users/{id}"]
}
```
→ Can register full agent instances + agentCardManifest ✅✅
→ Supports agent identity creation, A2A protocol, etc. ✅✅

### Required Permissions to Create Blueprint:
- `AgentIdentityBlueprint.Create` (requires **Global Administrator** or **Privileged Role Administrator**)
- Requires specific roles: **Agent ID Administrator** or **Agent ID Developer**

### Next Steps to Enable Full Registry Features:

**Option 1: Create Agent Identity Blueprint (Recommended)**
1. Create blueprint using POST to `/beta/applications/graph.agentIdentityBlueprint`
2. Create blueprint principal via POST to `/beta/serviceprincipals/graph.agentIdentityBlueprintPrincipal`
3. Configure credentials (managed identity or client secret)
4. Define OAuth scopes for your agent
5. Re-register agent with blueprint's identity

**Option 2: Continue with Current Approach (Limitations)**
- Use operational registry features only (instance registration, query, discovery)
- Store agent card metadata externally (custom database/API)
- Wait for Microsoft to potentially expand permissions for standard app registrations

### Documentation References:
- [Create Agent Identity Blueprint](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/create-blueprint?tabs=microsoft-graph-api)
- [Register Agents to Registry](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/publish-agents-to-registry?tabs=Microsoft-Entra-admin-center)

**Conclusion:** Our 403 errors are expected behavior. The `agentCardManifest` field requires an **Agent Identity Blueprint**, not a standard app registration.

## 1g) BONUS: Create Agent Identity Blueprint (Requires Admin Permissions)

This cell demonstrates how to create a proper **Agent Identity Blueprint** for full registry capabilities. 

⚠️ **Prerequisites:**
- Must have **Global Administrator** or **Privileged Role Administrator** role
- Requires `AgentIdentityBlueprint.Create` permission
- User must have **Agent ID Administrator** or **Agent ID Developer** role assigned
- **Required:** Add `USER_ID=<your-user-object-id>` to your `.env` file

**Why USER_ID is needed:**
- Blueprints require user sponsors and owners (not service principals)
- Service principal tokens (app-only auth) cannot access `/me` endpoint
- You must explicitly provide a user object ID

**To get your user ID:**
```bash
# Using Azure CLI
az ad signed-in-user show --query id -o tsv

# OR via PowerShell
(Get-AzADUser -UserPrincipalName your.email@company.com).Id
```

If you don't have these permissions, this cell will fail with 403 Forbidden - which is expected!

In [9]:
# STEP 1: Create Agent Identity Blueprint Application
# This requires Global Admin or Privileged Role Admin permissions

import uuid

token = get_graph_token()
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
    "OData-Version": "4.0"
}

print("=== Creating Agent Identity Blueprint ===")
print("⚠️  This requires elevated admin permissions\n")

# Note: Service principals (app-only auth) cannot use /me endpoint
# You need to specify a user ID for sponsors and owners
# Option 1: Set USER_ID in .env file
# Option 2: Use interactive (delegated) authentication instead

USER_ID = os.getenv("USER_ID")  # Add this to your .env file

if not USER_ID:
    print("❌ USER_ID not found in environment variables")
    print("\nAgent Identity Blueprints require user sponsors and owners.")
    print("However, service principal (app-only) tokens cannot access /me endpoint.\n")
    print("Solution: Add your user ID to .env file:")
    print("  USER_ID=<your-user-object-id>\n")
    print("To find your user ID, run this command with user credentials:")
    print("  az ad signed-in-user show --query id -o tsv")
    print("\nOR use the interactive_agent.py for delegated permissions.")
    raise ValueError("USER_ID required for blueprint creation with service principal")

print(f"Using User ID from environment: {USER_ID}")
print("Note: This user will be set as sponsor and owner of the blueprint\n")

# Optionally verify the user exists (requires User.Read.All permission)
try:
    user_response = requests.get(
        f"https://graph.microsoft.com/v1.0/users/{USER_ID}",
        headers={"Authorization": f"Bearer {token}"},
        timeout=30
    )
    
    if user_response.status_code == 200:
        current_user = user_response.json()
        user_name = current_user.get('displayName', 'Unknown')
        user_upn = current_user.get('userPrincipalName', 'Unknown')
        
        print(f"✓ User found: {user_name} ({user_upn})")
        print(f"  User ID: {USER_ID}\n")
        user_id = USER_ID
        
        # Create Agent Identity Blueprint
        blueprint_payload = {
            "@odata.type": "Microsoft.Graph.AgentIdentityBlueprint",
            "displayName": "Autonomous Agent Demo Blueprint",
            "sponsors@odata.bind": [
                f"https://graph.microsoft.com/v1.0/users/{user_id}"
            ],
            "owners@odata.bind": [
                f"https://graph.microsoft.com/v1.0/users/{user_id}"
            ]
        }
        
        print("Creating blueprint application...")
        print(json.dumps(blueprint_payload, indent=2)[:400] + "...\n")
        
        blueprint_response = requests.post(
            "https://graph.microsoft.com/beta/applications/graph.agentIdentityBlueprint",
            headers=headers,
            json=blueprint_payload,
            timeout=30
        )
        
        if blueprint_response.status_code in [200, 201]:
            blueprint = blueprint_response.json()
            blueprint_app_id = blueprint.get('appId')
            blueprint_object_id = blueprint.get('id')
            
            print("✓✓✓ Agent Identity Blueprint Created! ✓✓✓")
            print(f"  App ID (Client ID): {blueprint_app_id}")
            print(f"  Object ID: {blueprint_object_id}")
            print(f"  Display Name: {blueprint.get('displayName')}")
            
            # STEP 2: Create Service Principal for the Blueprint
            print("\n--- Creating Blueprint Service Principal ---")
            
            principal_payload = {
                "appId": blueprint_app_id
            }
            
            principal_response = requests.post(
                "https://graph.microsoft.com/beta/serviceprincipals/graph.agentIdentityBlueprintPrincipal",
                headers=headers,
                json=principal_payload,
                timeout=30
            )
            
            if principal_response.status_code in [200, 201]:
                principal = principal_response.json()
                principal_id = principal.get('id')
                
                print("✓ Service Principal Created!")
                print(f"  Principal ID: {principal_id}")
                
                # STEP 3: Add client secret
                print("\n--- Adding Client Secret ---")
                
                secret_payload = {
                    "passwordCredential": {
                        "displayName": "Agent Demo Secret",
                        "endDateTime": "2027-12-31T23:59:59Z"
                    }
                }
                
                secret_response = requests.post(
                    f"https://graph.microsoft.com/beta/applications/{blueprint_object_id}/addPassword",
                    headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
                    json=secret_payload,
                    timeout=30
                )
                
                if secret_response.status_code in [200, 201]:
                    secret_data = secret_response.json()
                    client_secret = secret_data.get('secretText')
                    
                    print("✓ Client Secret Created!")
                    print(f"  ⚠️  SAVE THIS SECRET (shown only once):")
                    print(f"  {client_secret}")
                    
                    # STEP 4: Configure OAuth scope
                    print("\n--- Configuring OAuth Scope ---")
                    
                    scope_id = str(uuid.uuid4())
                    scope_payload = {
                        "identifierUris": [f"api://{blueprint_app_id}"],
                        "api": {
                            "oauth2PermissionScopes": [
                                {
                                    "adminConsentDescription": "Allow the application to access the agent on behalf of the signed-in user.",
                                    "adminConsentDisplayName": "Access agent",
                                    "id": scope_id,
                                    "isEnabled": True,
                                    "type": "User",
                                    "value": "access_agent"
                                }
                            ]
                        }
                    }
                    
                    scope_response = requests.patch(
                        f"https://graph.microsoft.com/beta/applications/{blueprint_object_id}",
                        headers=headers,
                        json=scope_payload,
                        timeout=30
                    )
                    
                    if scope_response.status_code in [200, 204]:
                        print("✓ OAuth Scope Configured!")
                        print(f"  Scope: access_agent")
                        print(f"  Identifier URI: api://{blueprint_app_id}")
                        
                        print("\n" + "="*70)
                        print("✓✓✓ Agent Identity Blueprint Fully Configured! ✓✓✓")
                        print("\nUpdate your .env file with these values:")
                        print(f"BLUEPRINT_CLIENT_ID={blueprint_app_id}")
                        print(f"BLUEPRINT_CLIENT_SECRET={client_secret}")
                        print(f"BLUEPRINT_OBJECT_ID={blueprint_object_id}")
                        print("\nNow you can register agents with agentCardManifest using this blueprint!")
                        
                    else:
                        print(f"✗ OAuth scope configuration failed: {scope_response.status_code}")
                        print(f"  Response: {scope_response.text}")
                else:
                    print(f"✗ Secret creation failed: {secret_response.status_code}")
                    print(f"  Response: {secret_response.text}")
                    
            else:
                print(f"✗ Service principal creation failed: {principal_response.status_code}")
                error_data = principal_response.json()
                if 'error' in error_data:
                    print(f"  Error: {error_data['error'].get('message')}")
                    
        else:
            print(f"✗ Blueprint creation failed: {blueprint_response.status_code}")
            error_data = blueprint_response.json()
            
            if 'error' in error_data:
                error = error_data['error']
                print(f"  Error Code: {error.get('code')}")
                print(f"  Message: {error.get('message')}")
                
                if blueprint_response.status_code == 403:
                    print("\n⚠️  EXPECTED: Insufficient Permissions")
                    print("  You need one of these roles:")
                    print("    - Global Administrator")
                    print("    - Privileged Role Administrator")
                    print("\n  Plus this Graph permission:")
                    print("    - AgentIdentityBlueprint.Create")
                    print("\n  This is why agentCardManifest registration failed earlier.")
                    print("  Contact your tenant admin to create the blueprint.")
    else:
        print(f"⚠️  Could not verify user: {user_response.status_code}")
        print("  Proceeding with USER_ID from environment anyway...")
        print("  (User.Read.All permission needed to verify)\n")
        user_id = USER_ID
        
except ValueError as ve:
    # USER_ID not set - already handled above
    print(f"\n✗ {ve}")
except Exception as e:
    print(f"✗ Exception: {e}")
    import traceback
    traceback.print_exc()

=== Creating Agent Identity Blueprint ===
⚠️  This requires elevated admin permissions

Using User ID from environment: 3f1c2b34-07f7-4865-8dca-b4d9bc583fc8
Note: This user will be set as sponsor and owner of the blueprint

⚠️  Could not verify user: 403
  Proceeding with USER_ID from environment anyway...
  (User.Read.All permission needed to verify)



## 2) Query the Registry — Discover All Agents

In [10]:
token = get_graph_token()
headers = {"Authorization": f"Bearer {token}"}

url = "https://graph.microsoft.com/beta/agentRegistry/agentInstances"

print("=== Querying Agent Registry ===")
print(f"GET {url}\n")

try:
    response = requests.get(url, headers=headers, timeout=30)
    
    if response.status_code == 200:
        result = response.json()
        agents = result.get("value", [])
        
        print(f"✓ Found {len(agents)} agent(s) in registry\n")
        
        for idx, agent in enumerate(agents, 1):
            print(f"--- Agent {idx} ---")
            print(f"  ID: {agent.get('id')}")
            print(f"  Display Name: {agent.get('displayName')}")
            print(f"  Description: {agent.get('description', 'N/A')}")
            print(f"  URL: {agent.get('url', 'N/A')}")
            print(f"  Agent Identity ID: {agent.get('agentIdentityId', 'N/A')}")
            print()
    else:
        print(f"✗ Query failed: {response.status_code}")
        print(f"  Error: {response.text}")
        
        if response.status_code == 403:
            print("\n⚠ Permission Issue:")
            print("  Your service principal needs 'AgentRegistry.Read.All' permission.")
            
except Exception as e:
    print(f"✗ Exception during query: {e}")

=== Querying Agent Registry ===
GET https://graph.microsoft.com/beta/agentRegistry/agentInstances

✓ Found 0 agent(s) in registry



## 3) Search by Skills — Find Agents with Specific Capabilities

The registry supports OData filtering to search agents by skills, capabilities, or other metadata.

In [11]:
# Search for agents with "storage" skill
search_term = "storage"

token = get_graph_token()
headers = {"Authorization": f"Bearer {token}"}

# Note: The exact filter syntax may vary based on API version
# This is a simplified search - adjust based on actual API capabilities
url = f"https://graph.microsoft.com/beta/agentRegistry/agentInstances?$filter=contains(displayName,'{search_term}') or contains(description,'{search_term}')"

print(f"=== Searching for Agents with '{search_term}' ===")
print(f"Query: {url}\n")

try:
    response = requests.get(url, headers=headers, timeout=30)
    
    if response.status_code == 200:
        result = response.json()
        agents = result.get("value", [])
        
        print(f"✓ Found {len(agents)} matching agent(s)\n")
        
        for idx, agent in enumerate(agents, 1):
            print(f"--- Match {idx} ---")
            print(f"  Display Name: {agent.get('displayName')}")
            print(f"  Description: {agent.get('description', 'N/A')[:100]}...")
            
            # Try to extract skills from agent card if available
            card = agent.get('agentCardManifest', {})
            skills = card.get('skills', [])
            if skills:
                print(f"  Skills:")
                for skill in skills[:3]:  # Show first 3
                    print(f"    - {skill.get('name')}: {skill.get('description', '')[:60]}...")
            print()
    else:
        print(f"✗ Search failed: {response.status_code}")
        print(f"  Error: {response.text}")
        
        if "filter" in response.text.lower():
            print("\n⚠ Filter may not be supported yet (API in preview).")
            print("  Try the basic query without filters instead.")
            
except Exception as e:
    print(f"✗ Exception during search: {e}")

=== Searching for Agents with 'storage' ===
Query: https://graph.microsoft.com/beta/agentRegistry/agentInstances?$filter=contains(displayName,'storage') or contains(description,'storage')

✗ Search failed: 400
  Error: {"error":{"code":"BadRequest","message":"Invalid filter clause: Could not find a property named 'description' on type 'microsoft.graph.agentInstance'.","innerError":{"date":"2026-01-20T02:46:20","request-id":"9793902e-a24e-41c8-9d4e-22fbd1587a84","client-request-id":"9793902e-a24e-41c8-9d4e-22fbd1587a84"}}}

⚠ Filter may not be supported yet (API in preview).
  Try the basic query without filters instead.


## 4) Get Agent Details — Including Full Metadata

In [12]:
# First, get the list of agents to find an ID
token = get_graph_token()
headers = {"Authorization": f"Bearer {token}"}

url = "https://graph.microsoft.com/beta/agentRegistry/agentInstances"
response = requests.get(url, headers=headers, timeout=30)

if response.status_code == 200:
    result = response.json()
    agents = result.get("value", [])
    
    if agents:
        # Get details for the first agent
        first_agent_id = agents[0].get('id')
        
        print(f"=== Getting Full Details for Agent ===")
        print(f"Agent ID: {first_agent_id}\n")
        
        detail_url = f"https://graph.microsoft.com/beta/agentRegistry/agentInstances/{first_agent_id}"
        detail_response = requests.get(detail_url, headers=headers, timeout=30)
        
        if detail_response.status_code == 200:
            agent_details = detail_response.json()
            
            print("✓ Agent Details:")
            print(json.dumps(agent_details, indent=2)[:1000] + "...")
            
            # Extract and display specific metadata
            card = agent_details.get('agentCardManifest', {})
            
            if card:
                print("\n=== Extracted Metadata ===")
                print(f"Version: {card.get('version')}")
                print(f"Provider: {card.get('provider', {}).get('name')}")
                
                skills = card.get('skills', [])
                print(f"\nSkills ({len(skills)}):")
                for skill in skills:
                    print(f"  - {skill.get('name')}: {skill.get('description')}")
                
                print(f"\nInput Modes: {', '.join(card.get('defaultInputModes', []))}")
                print(f"Output Modes: {', '.join(card.get('defaultOutputModes', []))}")
                
                if 'securitySchemes' in card:
                    print(f"\nSecurity Schemes: {list(card['securitySchemes'].keys())}")
        else:
            print(f"✗ Failed to get details: {detail_response.status_code}")
            print(f"  Error: {detail_response.text}")
    else:
        print("No agents found in registry to inspect.")
else:
    print(f"✗ Failed to list agents: {response.status_code}")
    print(f"  Error: {response.text}")

No agents found in registry to inspect.


## 5) Collections — Discovery Boundaries (Read-Only)

Collections control which agents can discover each other. Only agents in the same collection can see each other during discovery queries.

**Note:** Collection management typically requires admin permissions. This cell attempts read-only access.

In [13]:
token = get_graph_token()
headers = {"Authorization": f"Bearer {token}"}

# Try to list collections (may not be available in current preview)
url = "https://graph.microsoft.com/beta/agentRegistry/collections"

print("=== Attempting to List Collections ===")
print(f"GET {url}\n")

try:
    response = requests.get(url, headers=headers, timeout=30)
    
    if response.status_code == 200:
        result = response.json()
        collections = result.get("value", [])
        
        print(f"✓ Found {len(collections)} collection(s)\n")
        
        for idx, collection in enumerate(collections, 1):
            print(f"--- Collection {idx} ---")
            print(f"  ID: {collection.get('id')}")
            print(f"  Name: {collection.get('displayName', collection.get('name', 'N/A'))}")
            print(f"  Description: {collection.get('description', 'N/A')}")
            print()
    elif response.status_code == 404:
        print("⚠ Collections endpoint not available yet (API in preview).")
        print("  This feature may be added in future updates.")
    elif response.status_code == 403:
        print("✗ Insufficient permissions to list collections.")
        print("  Typically requires admin-level access.")
    else:
        print(f"✗ Query failed: {response.status_code}")
        print(f"  Error: {response.text}")
        
except Exception as e:
    print(f"✗ Exception: {e}")

print("\n=== Collection Concepts ===")
print("Collections provide discovery boundaries:")
print("  - Agents must be explicitly assigned to collections")
print("  - Agents can only discover other agents in their collections")
print("  - Enables Zero Trust and least-privilege discovery")
print("  - Supports both built-in and custom collections")

=== Attempting to List Collections ===
GET https://graph.microsoft.com/beta/agentRegistry/collections

✗ Query failed: 400
  Error: {"error":{"code":"BadRequest","message":"Resource not found for the segment 'collections'.","innerError":{"date":"2026-01-20T02:46:21","request-id":"c9d5ba7a-6a5a-48e2-a065-742eb41f2715","client-request-id":"c9d5ba7a-6a5a-48e2-a065-742eb41f2715"}}}

=== Collection Concepts ===
Collections provide discovery boundaries:
  - Agents must be explicitly assigned to collections
  - Agents can only discover other agents in their collections
  - Enables Zero Trust and least-privilege discovery
  - Supports both built-in and custom collections


---

## Summary — What We Demonstrated

**Agent Registry is the "directory service for agents":**

1. ✓ **Registration** — Agents publish metadata (skills, capabilities, protocols) to the registry
2. ✓ **Discovery** — Agents query the registry to find other agents based on capabilities
3. ✓ **Metadata** — Rich agent cards include skills, security schemes, endpoints, and more
4. ✓ **Governance** — Collections enforce discovery boundaries (Zero Trust)

**Real-World Use Cases:**
- **Agent-to-Agent (A2A)** orchestration: HR agent discovers payroll agent for delegation
- **Shadow AI prevention**: Track all deployed agents across the organization
- **Skill-based routing**: Route user requests to agents with the right capabilities
- **Compliance**: Audit which agents discovered/called which other agents

**Next Steps:**
- Implement A2A protocol communication between discovered agents
- Set up collection-based access policies
- Integrate with Conditional Access for risk-based discovery
- Build orchestration layer using registry queries

**API Reference:**
- [Agent Registry Graph API](https://learn.microsoft.com/en-us/graph/api/resources/agentregistry-overview)
- [Register Agents](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/publish-agents-to-registry)
- [A2A Protocol](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/registry-agent-to-agent-protocol)